In [1]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer, *-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.25323.1

Loading extension script from `C:\Users\Pedro_Carvalho1\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.25323.1\interactive-extensions\dotnet\extension.dib`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [2]:
using System.IO;

var connStr = File.ReadAllText(@"C:\Users\Pedro_Carvalho1\conn_str.txt");

#!share --from csharp connStr

In [3]:
#!connect mssql --kernel-name sql --connection-string @connStr

Kernel added: #!sql-sql

In [7]:
#!sql-sql

-- Garantir que o schema existe
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'sql009')
    EXEC('CREATE SCHEMA sql009');
GO

-- Dropar se já existir
IF OBJECT_ID('sql009.Customers', 'U') IS NOT NULL DROP TABLE sql009.Customers;
IF OBJECT_ID('sql009.Orders', 'U') IS NOT NULL DROP TABLE sql009.Orders;
IF OBJECT_ID('sql009.OrderItems', 'U') IS NOT NULL DROP TABLE sql009.OrderItems;
GO

-- Customers
CREATE TABLE sql009.Customers (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(100) NOT NULL,
    region VARCHAR(50) NOT NULL
);

-- Orders
CREATE TABLE sql009.Orders (
    order_id INT PRIMARY KEY,
    customer_id INT NOT NULL,
    order_date DATE NOT NULL,
    status VARCHAR(20) NOT NULL, -- ex: Completed, Pending, Cancelled
    CONSTRAINT FK_Orders_Customers FOREIGN KEY (customer_id) REFERENCES sql009.Customers(customer_id)
);

-- OrderItems
CREATE TABLE sql009.OrderItems (
    order_item_id INT PRIMARY KEY,
    order_id INT NOT NULL,
    product_id INT NOT NULL,
    quantity INT NOT NULL,
    unit_price DECIMAL(10,2) NOT NULL,
    discount DECIMAL(5,2) NOT NULL, -- percentual de desconto (ex: 0.10 = 10%)
    CONSTRAINT FK_OrderItems_Orders FOREIGN KEY (order_id) REFERENCES sql009.Orders(order_id)
);
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

In [8]:
#!sql-sql

-- Customers
INSERT INTO sql009.Customers (customer_id, customer_name, region) VALUES
(1, 'Alice', 'Europe'),
(2, 'Bob', 'North America'),
(3, 'Charlie', 'Europe'),
(4, 'Diana', 'South America');

-- Orders
INSERT INTO sql009.Orders (order_id, customer_id, order_date, status) VALUES
(101, 1, '2024-01-10', 'Completed'),
(102, 1, '2024-02-15', 'Completed'),
(103, 2, '2024-01-20', 'Cancelled'),
(104, 2, '2024-03-05', 'Completed'),
(105, 3, '2024-02-01', 'Pending'),
(106, 4, '2024-01-25', 'Completed'),
(107, 4, '2024-03-02', 'Completed');

-- OrderItems
INSERT INTO sql009.OrderItems (order_item_id, order_id, product_id, quantity, unit_price, discount) VALUES
(1001, 101, 501, 2, 50.00, 0.00),
(1002, 101, 502, 1, 200.00, 0.10),
(1003, 102, 501, 3, 50.00, 0.05),
(1004, 103, 503, 5, 30.00, 0.00),
(1005, 104, 504, 10, 15.00, 0.20),
(1006, 105, 505, 1, 1000.00, 0.00),
(1007, 106, 506, 4, 25.00, 0.00),
(1008, 107, 502, 2, 200.00, 0.15),
(1009, 107, 507, 1, 500.00, 0.00);
GO


(4 rows affected)

(7 rows affected)

(9 rows affected)

❓ Enunciado da Questão

Escreva uma query que retorne, para cada região, o valor médio de gasto por cliente considerando apenas pedidos com status = 'Completed'.

O valor gasto por cliente deve considerar a soma de todos os seus pedidos finalizados, calculando corretamente o desconto aplicado em cada item.

Exclua clientes que não possuem nenhum pedido finalizado.

Arredonde o resultado final para duas casas decimais.

Resultado esperado (colunas):

region

avg_customer_spend

In [10]:
#!sql-sql
select * from sql009.Customers;
select * from sql009.Orders;
select * from sql009.OrderItems;

(4 rows affected)

(7 rows affected)

(9 rows affected)

customer_id,customer_name,region
1,Alice,Europe
2,Bob,North America
3,Charlie,Europe
4,Diana,South America


order_id,customer_id,order_date,status
101,1,2024-01-10 00:00:00Z,Completed
102,1,2024-02-15 00:00:00Z,Completed
103,2,2024-01-20 00:00:00Z,Cancelled
104,2,2024-03-05 00:00:00Z,Completed
105,3,2024-02-01 00:00:00Z,Pending
106,4,2024-01-25 00:00:00Z,Completed
107,4,2024-03-02 00:00:00Z,Completed


order_item_id,order_id,product_id,quantity,unit_price,discount
1001,101,501,2,50.00,0.00
1002,101,502,1,200.00,0.10
1003,102,501,3,50.00,0.05
1004,103,503,5,30.00,0.00
1005,104,504,10,15.00,0.20
1006,105,505,1,1000.00,0.00
1007,106,506,4,25.00,0.00
1008,107,502,2,200.00,0.15
1009,107,507,1,500.00,0.00


In [25]:
#!sql-sql

-- Escreva uma query que retorne, para cada região, o valor médio de gasto por cliente considerando apenas pedidos com status = 'Completed'.
-- O valor gasto por cliente deve considerar a soma de todos os seus pedidos finalizados, calculando corretamente o desconto aplicado em cada item.
-- Exclua clientes que não possuem nenhum pedido finalizado.
-- Arredonde o resultado final para duas casas decimais.

SELECT  
    c.region
    ,
    CAST(
        ROUND(
            1.0 *
            AVG( ( oi.quantity * ( oi.unit_price * ( 1 - oi.discount ) ) * 1.0 ) )
        ,2 ) 
    AS DECIMAL(10,2)) as avg_res
FROM 
    sql009.Customers c
LEFT JOIN
    sql009.Orders o on c.customer_id = o.customer_id
LEFT JOIN
    sql009.OrderItems oi on o.order_id = oi.order_id
WHERE
    o.status = 'Completed'    
GROUP BY
    c.region

     



(3 rows affected)

region,avg_res
Europe,140.83
North America,120.00
South America,313.33


In [35]:
#!sql-sql

WITH _AVG_CUST AS (
    SELECT  
        C.CUSTOMER_ID
        , C.CUSTOMER_NAME
        , C.REGION
        , SUM( ( oi.quantity * ( oi.unit_price * ( 1 - oi.discount ) ) * 1.0 ) ) AS AVG_CUST
    FROM    
        SQL009.Customers C
    INNER JOIN
        sql009.Orders o on c.customer_id = o.customer_id
    INNER JOIN
        sql009.OrderItems oi on o.order_id = oi.order_id
    WHERE
        o.status = 'Completed'
    GROUP BY
        C.CUSTOMER_ID
        , C.CUSTOMER_NAME
        , C.REGION
)
SELECT
    REGION
    , CAST( ROUND( 1.0 * AVG( AVG_CUST ) , 2 ) AS DECIMAL(10,2) ) AS avg_customer_spend 
FROM _AVG_CUST
GROUP BY REGION

(3 rows affected)

REGION,avg_customer_spend
Europe,422.50
North America,120.00
South America,940.00
